# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [4]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [5]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [6]:
from key import api_key, api_secret

In [7]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback=False)

# Creating Datafram
- Basing this strategy in a 6 month change

In [8]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [9]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [10]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [11]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

### Sorting by biggest increases

In [12]:
momentum_df.sort_values(by='Net Change', ascending=False, inplace=True)

In [13]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
360,PLTR,44.86,93.3768,108.15,N/a
308,MCK,507.41,697.7600,37.51,N/a
171,EQT,37.48,51.2600,36.77,N/a
468,VRSN,181.26,246.9500,36.24,N/a
433,TTWO,161.79,213.1900,31.77,N/a


# Creating Excel Output with all data

In [14]:
momentum_df_copy = momentum_df.copy()

In [15]:
momentum_df_copy = momentum_df_copy.drop('Number of Shares to Buy', axis=1)
momentum_df_copy.head()

,Ticker,Price (6 months),Price (now),Net Change
360,PLTR,44.86,93.3768,108.15
308,MCK,507.41,697.7600,37.51
171,EQT,37.48,51.2600,36.77
468,VRSN,181.26,246.9500,36.24
433,TTWO,161.79,213.1900,31.77


In [16]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

momentum_df_copy.to_excel(writer, sheet_name='Movers', index=False)

### Formatting

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [18]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [19]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying Formats

In [20]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format]
}

In [21]:
for column in column_formats.keys():
    writer.sheets['Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [22]:
writer.close()

# Keeping Top 50 Movers

In [23]:
final_df = momentum_df.copy()

In [24]:
final_df = final_df.head(50)
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 360 to 339
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price (6 months)         50 non-null     float64
 2   Price (now)              50 non-null     float64
 3   Net Change               50 non-null     float64
 4   Number of Shares to Buy  50 non-null     object 
dtypes: float64(3), object(2)
memory usage: 2.3+ KB


In [25]:
final_df.reset_index(inplace=True)
final_df.drop('index', axis = 1, inplace=True)
final_df

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.3768,108.15,N/a
1,MCK,507.41,697.7600,37.51,N/a
2,EQT,37.48,51.2600,36.77,N/a
3,VRSN,181.26,246.9500,36.24,N/a
4,TTWO,161.79,213.1900,31.77,N/a
5,PAYC,164.63,216.6600,31.60,N/a
6,TKO,115.25,149.0450,29.32,N/a
7,NFLX,754.68,971.1500,28.68,N/a
8,CNP,29.31,37.3650,27.48,N/a
9,EXE,84.79,107.1400,26.36,N/a


# Calculating the Number of Shares to Buy

In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the porfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Invalid input! \n Please try again.')
        portfolio_size = input('Enter the size of the porfolio: ')


In [27]:
portfolio_input()

In [28]:
print(portfolio_size)

100000


In [29]:
position_size = float(portfolio_size) / len(final_df.index)
position_size

2000.0

In [30]:
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price (now)'])

In [31]:
final_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.3768,108.15,21
1,MCK,507.41,697.7600,37.51,2
2,EQT,37.48,51.2600,36.77,39
3,VRSN,181.26,246.9500,36.24,8
4,TTWO,161.79,213.1900,31.77,9


# Exporting to Excel

In [32]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [33]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [34]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [37]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format],
    'E': ['Number of Shares to Buy', integer_format]
}

In [38]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [39]:
writer.close()

# More Realistic Momentum Strategy

Want to differentiate from high quality and low quality
- high quality - have slow and steady outperformance over long period of time
- low quality - might be from one surge of increase 

We want high quality momentum stocks because it shows repeated success rather than a surge in increase due to one event.

In [40]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
hqm_df

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile


## Making API Calls

In [41]:
today = datetime.today()
one_year = today - relativedelta(years = 1, months = 1)

In [42]:
print(today)
print(one_year)

2025-04-17 15:22:11.872680
2024-03-17 15:22:11.872680


In [43]:
symbol = 'AAPL'
price_history = client.price_history(symbol, periodType='year', frequencyType='monthly', frequency=1, startDate=one_year, endDate=today).json()

In [44]:
print(price_history['candles'][6]['datetime'])

1727758800000


In [45]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', frequency=1, startDate=one_year, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_12_months = price_history['candles'][0]['close']
    price_6_months = price_history['candles'][6]['close']
    price_3_months = price_history['candles'][9]['close']
    price_1_month = price_history['candles'][-2]['close']
    net_change_12= round(((price_now - price_12_months)/ price_12_months) * 100, 2)
    net_change_6 = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    net_change_3 = round(((price_now - price_3_months)/ price_3_months) * 100, 2)
    net_change_1 = round(((price_now - price_1_month)/ price_1_month) * 100, 2)
    entries.append(
        [
            symbol,
            price_now,
            net_change_12,
            'N/a',
            net_change_6,
            'N/a',
            net_change_3,
            'N/a',
            net_change_1,
            'N/a'
        ]
    )

In [46]:
hqm_df = pd.DataFrame(entries, columns=hqm_columns)

In [47]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,130.7250,46.42,N/a,34.56,N/a,31.16,N/a,-3.84,N/a
1,AOS,63.1300,-29.29,N/a,-25.12,N/a,-25.54,N/a,-2.12,N/a
2,ABT,131.1000,18.57,N/a,23.80,N/a,26.12,N/a,3.33,N/a
3,ABBV,173.6050,-2.72,N/a,5.99,N/a,10.53,N/a,-0.83,N/a
4,ACN,283.0044,-16.15,N/a,-6.82,N/a,-5.98,N/a,-0.47,N/a


## Calculating Momentum Percentiles 

In [48]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for period in time_periods:
        hqm_df.loc[row, f'{period} Return Percentile'] = stats.percentileofscore(
            hqm_df[f'{period} Price Return'],
            hqm_df.loc[row, f'{period} Price Return']
        )

In [49]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,130.7250,46.42,95.029821,34.56,91.252485,31.16,90.059642,-3.84,6.461233
1,AOS,63.1300,-29.29,15.308151,-25.12,15.506958,-25.54,14.71173,-2.12,14.71173
2,ABT,131.1000,18.57,78.727634,23.80,83.101392,26.12,86.481113,3.33,85.188867
3,ABBV,173.6050,-2.72,48.111332,5.99,58.449304,10.53,69.681909,-0.83,29.025845
4,ACN,283.0044,-16.15,30.616302,-6.82,38.369781,-5.98,39.761431,-0.47,33.996024


## Calculating HQM Score
- HQM Score - mean of the four percentiles scores

In [50]:
from statistics import mean

In [51]:
for i in range(len(hqm_df)):
    momentum_percentiles = []
    for period in time_periods:
        momentum_percentiles.append(hqm_df.loc[i, f'{period} Return Percentile'])
    hqm_df.loc[i, 'HQM Score'] = mean(momentum_percentiles)

In [52]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,130.7250,46.42,95.029821,34.56,91.252485,31.16,90.059642,-3.84,6.461233,70.700795
1,AOS,63.1300,-29.29,15.308151,-25.12,15.506958,-25.54,14.71173,-2.12,14.71173,15.059642
2,ABT,131.1000,18.57,78.727634,23.80,83.101392,26.12,86.481113,3.33,85.188867,83.374751
3,ABBV,173.6050,-2.72,48.111332,5.99,58.449304,10.53,69.681909,-0.83,29.025845,51.317097
4,ACN,283.0044,-16.15,30.616302,-6.82,38.369781,-5.98,39.761431,-0.47,33.996024,35.685885


### Getting top performing stocks

In [53]:
hqm_df.sort_values(by='HQM Score', ascending=False, inplace=True)
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
360,PLTR,93.420,286.35,100.0,300.43,100.0,344.65,100.0,5.50,95.82505,98.956262
371,PM,163.840,80.28,99.204771,68.21,99.204771,63.95,99.204771,6.47,97.415507,98.757455
442,TPL,1295.830,134.77,99.602386,134.91,99.801193,115.52,99.801193,4.83,94.035785,98.310139
435,TRGP,175.870,58.58,97.017893,56.59,97.2167,53.89,98.011928,7.53,98.807157,97.763419
333,NFLX,970.445,54.53,96.620278,67.51,99.005964,50.05,97.2167,5.68,96.421471,97.316103


### exporting to excel

In [54]:
xl_path = Path('total_data.xlsx')

writer = pd.ExcelWriter(xl_path, engine='xlsxwriter')

hqm_df.to_excel(writer, sheet_name='Better Movers', index=False)

In [55]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [56]:
hqm_column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['One-Year Price Return', percent_format],
    'D': ['One-Year Return Percentile', percent_format],
    'E': ['Six-Month Price Return', percent_format],
    'F': ['Six-Month Return Percentile', percent_format],
    'G': ['Three-Month Price Return', percent_format],
    'H': ['Three-Month Return Percentile', percent_format],
    'I': ['One-Month Price Return', percent_format],
    'J': ['One-Month Return Percentile', percent_format],
    'K': ['HQM Score', percent_format]
}

In [57]:
for column in column_formats.keys():
    writer.sheets['Better Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Better Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [58]:
writer.close()